In [1]:
%cd ..

import importlib
import json
import math
import os
import socket
import sys
import time


internal_path = os.path.abspath(os.path.join('.'))
sys.path.append(internal_path)
sys.path.append(internal_path + '/datasets')
sys.path.append(internal_path + '/utils')

import datetime
import uuid
from argparse import ArgumentParser

import setproctitle
import torch
import numpy as np
import pandas as pd 
import json
import wandb

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import AutoMinorLocator
from matplotlib.collections import LineCollection



/pub/hofmann-scratch/glanzillo/ded


In [4]:
os.environ['WANDB_API_KEY'] = 'c7314b16e17009f66f3043c0b7968e1142054123'
#os.environ['WANDB_BASE_URL'] = 'https://api.wandb.ai/continually/DataEfficientDistillation'
#os.environ['WANDB_DISABLE_SERVICE'] = 'True' 
#os.environ['WANDB_CONFIG_DIR'] = "./wandb/"
#import wandb
wandb.login()

api = wandb.Api()
runs = api.runs(path='continually/DataEfficientDistillation', filters={'tags': 'useful'})
print("Loaded", len(runs), "runs")

Loaded 10 runs


In [5]:
dir(runs)

['QUERY',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_include_sweeps',
 '_load_page',
 '_sweeps',
 'client',
 'convert_objects',
 'cursor',
 'entity',
 'filters',
 'index',
 'last_response',
 'length',
 'more',
 'next',
 'objects',
 'order',
 'per_page',
 'project',
 'update_variables',
 'variables']

In [ ]:
runs.summary

In [10]:
runs.objects[0].history()

,_step,config,_runtime,_timestamp,parameter count,parameter count without embedding,eval_batches,eval_tokens,eval_bytes,_train/tokens_seen,...,_eval/normalised_ppl_over_tokens,_eval/normalised_loss,_eval/normalised_ppl_over_gpuseconds,x_axes/gpu_seconds,_time/tokens_per_second,_eval/normalised_loss_over_gpuseconds,_eval/ppl_over_gpuseconds,_train/loss,_time/iterations_per_second,_eval/top10_acc
0,0,{'_latest_artifact_path': 'wandb-client-artifa...,9.283063,1.705768e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,9.298753,1.705768e+09,27549696.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,9.298922,1.705768e+09,NaN,10772480.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,46.049116,1.705768e+09,NaN,NaN,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,46.049463,1.705768e+09,NaN,NaN,NaN,4096000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,34100,NaN,166216.963398,1.705934e+09,NaN,NaN,NaN,NaN,NaN,2.234843e+09,...,2.452931,0.897284,2.452931,134483.279336,13722.071990,0.897284,31.164516,0.557782,0.209382,0.667533
347,34200,NaN,166708.903767,1.705935e+09,NaN,NaN,NaN,NaN,NaN,2.241397e+09,...,2.450727,0.896385,2.450727,134877.646889,13559.820004,0.896385,31.057346,0.552357,0.206906,0.667706
348,34300,NaN,167199.201432,1.705935e+09,NaN,NaN,NaN,NaN,NaN,2.247950e+09,...,2.451409,0.896663,2.451409,135272.014442,13606.167077,0.896663,31.090470,0.555941,0.207614,0.667612
349,34400,NaN,167689.394901,1.705936e+09,NaN,NaN,NaN,NaN,NaN,2.254504e+09,...,2.452767,0.897217,2.452767,135666.381995,13606.551301,0.897217,31.156529,0.548975,0.207619,0.667507


In [22]:
dir(runs)

['QUERY',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_include_sweeps',
 '_load_page',
 '_sweeps',
 'client',
 'convert_objects',
 'cursor',
 'entity',
 'filters',
 'index',
 'last_response',
 'length',
 'more',
 'next',
 'objects',
 'order',
 'per_page',
 'project',
 'update_variables',
 'variables']

In [17]:
df = runs.objects[0].history()
df.iloc[0]['config']

{'_type': 'table-file',
 'artifact_path': 'wandb-client-artifact://yk4hyyhfmfq7rwnapn23oips1xj2gxrxqspjg8vhcq7ui6b9uad6quln7k6rvcjuzx6rewek5nhu9fnqms2kmoobu2xko9qicuwnhsrzn573lq05fd37nu7e42wh904j/config.table.json',
 'ncols': 1,
 'nrows': 1,
 'path': 'media/table/config_0_679092a815950efe093e.table.json',
 'sha256': '679092a815950efe093e9c4ff06e14151561f1ec7a823fcfe64663aee096b370',
 'size': 1390,
 '_latest_artifact_path': 'wandb-client-artifact://bfz0cc2b3ffa3p4dh6rycc796wsi8nsodcrq4ab5oe3ywjfyc7v5u0ajw04h8mjhxqdgo32odj3zjsl4hnynlkqai03nnm6nko1c8sqvpzutk5gdrlenlroz04aop3gr:latest/config.table.json'}

In [18]:
df.columns

Index(['_step', 'config', '_runtime', '_timestamp', 'parameter count',
       'parameter count without embedding', 'eval_batches', 'eval_tokens',
       'eval_bytes', '_train/tokens_seen', '_train/learning_rate_0',
       '_eval/total_loss', '_eval/top10_acc_over_gpuseconds', '_eval/ppl',
       '_eval/fa', '_eval/ppl_over_tokens',
       '_eval/normalised_loss_over_tokens', '_eval/top1_acc_over_tokens',
       'x_axes/n_tokens', '_time/forward', '_time/eval',
       '_eval/top1_acc_over_gpuseconds', '_eval/top10_acc_over_tokens',
       '_eval/cka', '_eval/loss', '_time/train_step', '_time/total_step',
       '_train/cka', '_time/backward', '_train/fa', '_eval/normalised_ppl',
       '_eval/top1_acc', '_time/load_batch',
       '_eval/normalised_ppl_over_tokens', '_eval/normalised_loss',
       '_eval/normalised_ppl_over_gpuseconds', 'x_axes/gpu_seconds',
       '_time/tokens_per_second', '_eval/normalised_loss_over_gpuseconds',
       '_eval/ppl_over_gpuseconds', '_train/loss',
     

In [ ]:
losses_dict = {}
xaxis='steps'
for run in runs:
    if "skipless" in run.metadata['args'][0]:
        if "parallel" in run.metadata['args'][0]:
            name = "SAS-P"
        else:
            name = "SAS"
    else:
        if "parallel" in run.metadata['args'][0]:
            name = "Parallel"
        else:
            name = "Pre-LN"
    losses=run._full_history(samples=10000)
    eval_losses = [loss for loss in losses if "_eval/ppl" in loss and loss['_eval/ppl'] is not None]
    eval_xaxis = [loss['_runtime']/3600  if xaxis == "runtime" else loss["train/global_step"] for loss in eval_losses]
    eval_losses = [loss['_eval/ppl'] for loss in eval_losses]
    losses_dict[name] = (eval_xaxis, eval_losses)